## Objetivo general
Este notebook tiene como objetivo principal la extracción automatizada de datos geoespaciales desde la plataforma OpenStreetMap (OSM) utilizando la API de Overpass. Se enfoca en recolectar información clave sobre infraestructura urbana en diversas ciudades de los Estados Unidos, incluyendo:

-Escuelas (school)
-Hospitales (hospital)
-Semáforos (traffic_signals)
-Cruces peatonales (crossing)

## Importación de librerías

In [1]:
import overpy
import pandas as pd
from tqdm import tqdm
import time
import os 
import glob
import ast

 ## Definición de ciudades y sus bounding boxes
 Se define un diccionario cities con 20 ciudades y sus respectivas bounding boxes (latitud mínima, longitud mínima, latitud máxima, longitud máxima).

Estas coordenadas geográficas delimitan las áreas donde se realizará la búsqueda de datos en OSM.

In [2]:
cities = {
    "Miami": [25.70, -80.30, 25.90, -80.10],
    "Houston": [29.60, -95.60, 30.10, -95.00],
    "Los_Angeles": [33.90, -118.60, 34.20, -118.10],
    "Charlotte": [35.10, -80.95, 35.35, -80.70],
    "Dallas": [32.65, -97.00, 32.90, -96.65],
    "Orlando": [28.40, -81.50, 28.65, -81.20],
    "Austin": [30.10, -97.90, 30.40, -97.60],
    "Raleigh": [35.70, -78.80, 36.00, -78.50],
    "Nashville": [36.00, -86.90, 36.25, -86.60],
    "Baton_Rouge": [30.35, -91.25, 30.55, -91.05],
    "Atlanta": [33.65, -84.55, 33.90, -84.30],
    "Sacramento": [38.45, -121.60, 38.70, -121.25],
    "Phoenix": [33.35, -112.20, 33.60, -111.90],
    "San_Diego": [32.70, -117.30, 32.90, -117.00],
    "Minneapolis": [44.90, -93.35, 45.05, -93.20],
    "Richmond": [37.45, -77.55, 37.60, -77.40],
    "Oklahoma_City": [35.35, -97.80, 35.60, -97.25],
    "Jacksonville": [30.10, -81.90, 30.45, -81.50],
    "Tucson": [32.10, -111.10, 32.30, -110.85],
    "Columbia": [33.90, -81.15, 34.10, -80.85],  # Asumiendo Columbia, SC
}


## Función para generar consulta Overpass

In [ ]:
# Inicializar la API de Overpass
api = overpy.Overpass()

# Función para crear la consulta
def create_query(minLat, minLon, maxLat, maxLon):
    return f"""
    [out:json][timeout:25];
    (
        node["amenity"="school"]({minLat},{minLon},{maxLat},{maxLon});
        node["amenity"="hospital"]({minLat},{minLon},{maxLat},{maxLon});
        node["highway"="traffic_signals"]({minLat},{minLon},{maxLat},{maxLon});
        node["highway"="crossing"]({minLat},{minLon},{maxLat},{maxLon});
    );
    out body;
    """


##  Extracción y guardado de datos
Función process_city_data():

-Ejecuta la consulta Overpass para cada ciudad.
-Procesa los nodos obtenidos en un DataFrame.
-Guarda el archivo .csv con los datos de esa ciudad.

Se recorre cada ciudad definida anteriormente y se extraen sus datos en un proceso automatizado.


In [ ]:

# Función para obtener y procesar los datos de una ciudad
def process_city_data(city, bbox):
    minLat, minLon, maxLat, maxLon = bbox
    print(f"\nQuerying {city} with bbox: {minLat}, {minLon}, {maxLat}, {maxLon}...")

    # Crear la consulta
    query = create_query(minLat, minLon, maxLat, maxLon)

    try:
        # Ejecutar la consulta y obtener los resultados
        result = api.query(query)

        # Procesar los nodos de la respuesta
        data = [
            {
                "id": node.id,
                "category": node.tags.get("amenity", node.tags.get("highway", "N/A")),
                "latitude": node.lat,
                "longitude": node.lon,
                "tags": str(node.tags)
            }
            for node in tqdm(result.nodes, desc=f"Processing nodes for {city}", leave=False)
        ]

        # Crear un DataFrame de pandas
        df = pd.DataFrame(data)

        # Definir la ruta del archivo CSV
        directory = r"C:\Users\ASUS\Desktop\proyecto03\data"
        if not os.path.exists(directory):
            os.makedirs(directory)
        csv_filename = os.path.join(directory, f"{city.lower()}_osm_info.csv")
        
        # Guardar el DataFrame en un archivo CSV
        df.to_csv(csv_filename, index=False)
        print(f"{city}: saved to '{csv_filename}'")

        # Pausa entre consultas para evitar sobrecargar el servidor
        time.sleep(5)
    
    except Exception as e:
        print(f"Error querying {city}: {e}")

# Iterar sobre las ciudades y procesar los datos
for city, bbox in cities.items():
    process_city_data(city, bbox)

## Conclusiones
El notebook realiza una extracción geoespacial completa y automatizada de elementos urbanos clave desde OpenStreetMap.

La estructura modular permite reutilizar el código fácilmente para otras ciudades o elementos.

Cada ciudad genera su propio archivo CSV, facilitando el análisis posterior y la trazabilidad de los datos.

Se incorporan buenas prácticas como el manejo de errores y las pausas entre consultas.

